# 続編：APIを使ってみよう！〜準備と実践編〜（ハンズオン用）

## APIキーの取得と設定

### Gemini API のキー取得
GeminiのAPIを呼び出すには、APIキーが必要です。Google AI Studioで発行したAPIキーを、Google Colabのシークレット機能を使って安全に管理します。

APIキーが正しく設定されていることを確認します。

In [ ]:
import os
from google.colab import userdata

# シークレットからAPIキーを取得
api_key = userdata.get("GOOGLE_API_KEY")

# 環境変数に設定
os.environ["GOOGLE_API_KEY"] = api_key

# 確認（キー自体は表示しない）
if api_key:
    print("✅ APIキーの設定が完了しました。")
    print(f"キーの先頭: {api_key[:10]}...")
else:
    print("❌ エラー: APIキーが設定されていません。")

APIキーが正しく動作するか、簡単なテストを行います。

In [ ]:
from google import genai

# クライアントの初期化
client = genai.Client()

# テストリクエスト
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="こんにちは！"
)

print("--- Gemini APIからの応答 ---")
print(response.text)
print("---------------------------")

### OpenWeatherMap API のキー取得
OpenWeatherMapは、世界中の天気情報を提供する無料のAPIサービスです。 例 2: 天気予報 API（データ取得と処理）　のセクションで使用します。

APIキーが正しく設定されていることを確認します。

In [ ]:
from google.colab import userdata
import os

# Colabの秘密鍵機能から取得
api_key = userdata.get("WEATHER_API_KEY")
os.environ["WEATHER_API_KEY"] = api_key

正しく天気情報が取得できることを確認します。

In [ ]:
import requests
import os

API_KEY = os.environ.get("WEATHER_API_KEY")
if not API_KEY:
    print("エラー: APIキーが設定されていません。")
    exit()

url = f"http://api.openweathermap.org/data/2.5/weather?q=Tokyo&appid={API_KEY}&units=metric&lang=ja"
response = requests.get(url)

if response.status_code == 200:
    print("✅ API接続成功！")
    data = response.json()
    print(f"都市: {data['name']}")
    print(f"気温: {data['main']['temp']}℃")
else:
    print(f"❌ エラー: {response.status_code}")
    print(response.text)

## プログラムからのAPI使用例

### 1: Gemini API（AI機能の利用 - 基本版）
- API クライアント：requestsライブラリ
- モデル名：gemini-2.5-flash

In [ ]:
import requests
import json
import os  # 環境変数からキーを読み込むために使用

# 1. 準備：APIキーとエンドポイントを定義
# (Colab環境の場合、os.environ['GEMINI_API_KEY']にキーが設定されている前提)
API_KEY = os.environ.get("GEMINI_API_KEY")
if not API_KEY:
    print("エラー: GEMINI_API_KEYが設定されていません。")
    # ここでは実行せずに終了
    exit()

ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# 2. キー設定：ヘッダーに認証情報を組み込む
HEADERS = {"Content-Type": "application/json"}
# APIキーはURLのクエリパラメータとして渡す
URL = f"{ENDPOINT}?key={API_KEY}"

# 3. データ準備：質問文をJSON形式の注文データとして作成
PROMPT = "Web APIとは何か、小学生にもわかるように簡潔に説明してください。"

PAYLOAD = {"contents": [{"parts": [{"text": PROMPT}]}]}

# 4. リクエスト送信：POSTメソッドでヘッダーと注文データ(本文)を添えて送信
print("APIにリクエストを送信中...")
response = requests.post(
    URL, headers=HEADERS, data=json.dumps(PAYLOAD)  # Pythonの辞書をJSON文字列に変換
)

# 5. 結果抽出：応答の処理
if response.status_code == 200:
    data = response.json()

    # 複雑なJSON構造からテキスト部分を抽出
    try:
        generated_text = data["candidates"][0]["content"]["parts"][0]["text"]
        print("\n--- AIからの応答 ---")
        print(generated_text)
        print("--------------------")
    except (KeyError, IndexError):
        print(
            "\nエラー: JSON構造からテキストを抽出できませんでした。応答を確認してください。"
        )
        # デバッグ用にJSON全体を表示しても良い
        # print(json.dumps(data, indent=2))
else:
    print(f"\nエラーが発生しました。ステータスコード: {response.status_code}")
    print("エラーメッセージ:", response.text)

APIにリクエストを送信中...

--- AIからの応答 ---
ウェブAPIは、インターネットを通じて他のコンピューターと「おしゃべり」するための「窓口」のようなものです。

例えば、レストランで料理を注文するとき、メニューを見て、ウェイターに注文を伝えますよね？

ウェブAPIは、メニューが「どんな情報がもらえるか、どんなお願いができるか」を示し、ウェイターが「情報をやり取りする役目」を果たします。

だから、アプリやウェブサイトは、ウェブAPIを使って、天気予報や地図、お店の情報などを、他のコンピューターから簡単にもらうことができるんです。

--------------------


### 2: 天気予報 API（OpenWeatherMap） を使用して実際の天気データを取得する
- API クライアント：requestsライブラリ
- エンドポイント：http://api.openweathermap.org/data/2.5/weather

In [ ]:
import requests
import json
import os

API_KEY_WEATHER = os.environ.get("WEATHER_API_KEY")
if not API_KEY_WEATHER:
    print("エラー: APIキーが設定されていません。")
    exit()

CITY_NAME = "Tokyo"
URL_BASE = "http://api.openweathermap.org/data/2.5/weather"

# 1. URL組み立て: パラメータを辞書で定義
params = {
    "q": CITY_NAME,
    "appid": API_KEY_WEATHER,
    "units": "metric",  # 温度を摂氏(℃)で取得するよう指定 (APIによって異なります)
    "lang": "ja",  # 日本語で取得するよう指定
}

print(f"APIにリクエストを送信中... (都市: {CITY_NAME})")

# 2. リクエスト送信: requests.get() を使ってURLとパラメータを送信
# requestsライブラリがパラメータを自動でURLに組み込んでくれます
try:
    response = requests.get(URL_BASE, params=params)
    response.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる

    # 3. データ解析: JSONデータを読み込む
    data = response.json()

    # 4. データ抽出と表示: JSONデータから必要な情報を取り出す
    temp_celsius = data["main"]["temp"]  # main > temp のラベルを辿る
    weather_desc = data["weather"][0][
        "description"
    ]  # weather (リストの0番目) > description を辿る

    print("\n--- 天気予報 APIからの応答 ---")
    print(f"都市: {data['name']}")
    print(f"天気: {weather_desc}")
    print(f"現在の気温: {temp_celsius:.1f} ℃")
    print("------------------------------")

except requests.exceptions.RequestException as e:
    print(f"\nエラー: APIアクセス中に問題が発生しました。キーやURLを確認してください。")
    print(f"詳細: {e}")
except KeyError:
    print("\nエラー: 応答JSONの形式が想定と異なります。データ構造を確認してください。")